# Model Tests

In [4]:
import numpy as np
import pandas as pd

import sklearn
import tensorflow as tf

In [5]:
df_prices = pd.read_csv('./data/prices.csv')

IndentationError: unexpected indent (<ipython-input-1-a66c0d46590f>, line 31)